# 如何使用少量示例

在本指南中，我们将学习如何创建一个简单的提示模板，在生成时为模型提供示例输入和输出。给定LLM几个这样的示例被称为少量示例（few-shotting），是一种简单而强大的引导生成的方法，在某些情况下可以显著提升模型性能。

少量示例提示模板可以从一组示例构建，或者通过一个[示例选择器](https://api.js.langchain.com/classes/langchain_core.example_selectors.BaseExampleSelector.html)类来构建，该类负责从已定义的示例集合中选择子集。

本指南将介绍使用字符串提示模板的少量示例方法。关于针对聊天模型使用聊天消息进行少量示例的指南，请参见[此处](/docs/how_to/few_shot_examples_chat/)。

:::info 预备知识

本指南假设您已熟悉以下概念：

- [提示模板](/docs/concepts/prompt_templates)
- [示例选择器](/docs/concepts/example_selectors)
- [LLMs](/docs/concepts/text_llms)
- [向量存储](/docs/concepts/#vectorstores)

:::

## 为少量示例创建一个格式化工具

配置一个将少量示例格式化为字符串的格式化工具。此格式化工具应为一个 `PromptTemplate` 对象。

In [1]:
import { PromptTemplate } from "@langchain/core/prompts";

const examplePrompt = PromptTemplate.fromTemplate("Question: {question}\n{answer}")

## 创建示例集

接下来，我们将创建一个少量示例的列表。每个示例应是一个字典，表示我们上面定义的格式化提示的示例输入。

In [2]:
const examples = [
    {
      question: "Who lived longer, Muhammad Ali or Alan Turing?",
      answer: `
  Are follow up questions needed here: Yes.
  Follow up: How old was Muhammad Ali when he died?
  Intermediate answer: Muhammad Ali was 74 years old when he died.
  Follow up: How old was Alan Turing when he died?
  Intermediate answer: Alan Turing was 41 years old when he died.
  So the final answer is: Muhammad Ali
  `
    },
    {
      question: "When was the founder of craigslist born?",
      answer: `
  Are follow up questions needed here: Yes.
  Follow up: Who was the founder of craigslist?
  Intermediate answer: Craigslist was founded by Craig Newmark.
  Follow up: When was Craig Newmark born?
  Intermediate answer: Craig Newmark was born on December 6, 1952.
  So the final answer is: December 6, 1952
  `
    },
    {
      question: "Who was the maternal grandfather of George Washington?",
      answer: `
  Are follow up questions needed here: Yes.
  Follow up: Who was the mother of George Washington?
  Intermediate answer: The mother of George Washington was Mary Ball Washington.
  Follow up: Who was the father of Mary Ball Washington?
  Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
  So the final answer is: Joseph Ball
  `
    },
    {
      question: "Are both the directors of Jaws and Casino Royale from the same country?",
      answer: `
  Are follow up questions needed here: Yes.
  Follow up: Who is the director of Jaws?
  Intermediate Answer: The director of Jaws is Steven Spielberg.
  Follow up: Where is Steven Spielberg from?
  Intermediate Answer: The United States.
  Follow up: Who is the director of Casino Royale?
  Intermediate Answer: The director of Casino Royale is Martin Campbell.
  Follow up: Where is Martin Campbell from?
  Intermediate Answer: New Zealand.
  So the final answer is: No
  `
    }
  ];

### 将示例和格式化器传递给 `FewShotPromptTemplate`

最后，创建一个 [`FewShotPromptTemplate`](https://api.js.langchain.com/classes/langchain_core.prompts.FewShotPromptTemplate.html) 对象。该对象接收少量样本示例和用于格式化这些示例的格式化器。当此 `FewShotPromptTemplate` 被格式化时，它会使用 `examplePrompt` 格式化传入的示例，然后将它们添加到最终提示中的 `suffix` 之前：

In [7]:
import { FewShotPromptTemplate } from "@langchain/core/prompts";

const prompt = new FewShotPromptTemplate({
    examples,
    examplePrompt,
    suffix: "Question: {input}",
    inputVariables: ["input"],
})

const formatted = await prompt.format({ input: "Who was the father of Mary Ball Washington?" })
console.log(formatted.toString())



Question: Who lived longer, Muhammad Ali or Alan Turing?

  Are follow up questions needed here: Yes.
  Follow up: How old was Muhammad Ali when he died?
  Intermediate answer: Muhammad Ali was 74 years old when he died.
  Follow up: How old was Alan Turing when he died?
  Intermediate answer: Alan Turing was 41 years old when he died.
  So the final answer is: Muhammad Ali
  

Question: When was the founder of craigslist born?

  Are follow up questions needed here: Yes.
  Follow up: Who was the founder of craigslist?
  Intermediate answer: Craigslist was founded by Craig Newmark.
  Follow up: When was Craig Newmark born?
  Intermediate answer: Craig Newmark was born on December 6, 1952.
  So the final answer is: December 6, 1952
  

Question: Who was the maternal grandfather of George Washington?

  Are follow up questions needed here: Yes.
  Follow up: Who was the mother of George Washington?
  Intermediate answer: The mother of George Washington was Mary Ball Washington.
  Follow

通过向模型提供这样的示例，我们可以引导模型生成更优的回答。

## 使用示例选择器

我们将重用前面章节中的示例集和格式化程序。不过，这次不会将示例直接提供给 `FewShotPromptTemplate` 对象，而是将它们提供给一个名为 [`SemanticSimilarityExampleSelector`](https://api.js.langchain.com/classes/langchain_core.example_selectors.SemanticSimilarityExampleSelector.html) 的 `ExampleSelector` 实现实例。此类会根据初始集合中示例与输入内容的相似性来选择少量示例。它使用嵌入模型计算输入与少量示例之间的相似性，并使用向量存储来进行最近邻搜索。

为了展示其效果，我们先单独初始化一个实例并调用它：

为嵌入模型设置您的OpenAI API密钥
```bash
export OPENAI_API_KEY="..."
```

In [14]:
import { SemanticSimilarityExampleSelector } from "@langchain/core/example_selectors";
import { MemoryVectorStore } from "langchain/vectorstores/memory";
import { OpenAIEmbeddings } from "@langchain/openai";

const exampleSelector = await SemanticSimilarityExampleSelector.fromExamples(
    // This is the list of examples available to select from.
    examples,
    // This is the embedding class used to produce embeddings which are used to measure semantic similarity.
    new OpenAIEmbeddings(),
    // This is the VectorStore class that is used to store the embeddings and do a similarity search over.
    MemoryVectorStore,
    {
        // This is the number of examples to produce.
        k: 1,
    }
)

// Select the most similar example to the input.
const question = "Who was the father of Mary Ball Washington?"
const selectedExamples = await exampleSelector.selectExamples({ question })
console.log(`Examples most similar to the input: ${question}`)
for (const example of selectedExamples) {
    console.log("\n");
    console.log(Object.entries(example).map(([k, v]) => `${k}: ${v}`).join("\n"))
}

Examples most similar to the input: Who was the father of Mary Ball Washington?


question: Who was the maternal grandfather of George Washington?
answer: 
  Are follow up questions needed here: Yes.
  Follow up: Who was the mother of George Washington?
  Intermediate answer: The mother of George Washington was Mary Ball Washington.
  Follow up: Who was the father of Mary Ball Washington?
  Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
  So the final answer is: Joseph Ball
  


现在，让我们创建一个 `FewShotPromptTemplate` 对象。该对象接收示例选择器和用于少量示例的格式化提示。

In [15]:
const prompt = new FewShotPromptTemplate({
    exampleSelector,
    examplePrompt,
    suffix: "Question: {input}",
    inputVariables: ["input"],
})

const formatted = await prompt.invoke({ input: "Who was the father of Mary Ball Washington?" });
console.log(formatted.toString())



Question: Who was the maternal grandfather of George Washington?

  Are follow up questions needed here: Yes.
  Follow up: Who was the mother of George Washington?
  Intermediate answer: The mother of George Washington was Mary Ball Washington.
  Follow up: Who was the father of Mary Ball Washington?
  Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
  So the final answer is: Joseph Ball
  

Question: Who was the father of Mary Ball Washington?


## 下一步

你现在已经学会了如何向提示中添加少量示例。

接下来，请查看本节中有关提示模板的其他操作指南、关于[使用聊天模型进行少量示例](/docs/how_to/few_shot_examples_chat)的相关操作指南，或[示例选择器操作指南](/docs/how_to/example_selectors/)的其他内容。